<a href="https://colab.research.google.com/github/Coonlee/three_sigma/blob/master/backtestFrameworkDraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 24 11:42:37 2018

@author: szq13821
"""

import matplotlib.pyplot as plt
from __future__ import division
from datetime import *
#import time
import numpy as np
import pandas as pd
import copy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import traceback
from sqlalchemy import create_engine
import datetime as dt
from datetime import timedelta 
from dateutil.relativedelta import relativedelta
import sys
import os
from random import *
import random
import copy
import gc


#读取onlineLearningOutput 
backtestInputDf = pd.DataFrame(onlineLearningOutput, columns = ['trade_date','ticker_symbol','model_suggested_weight'])

#读取每日每个标的股票的收益率的dataframe
backtestSymbolDailyReturnList = pd.DataFrame(......)

#此类每个对象模型的是出资人，在我们这个项目里不重要，可以忽略，使用时创建一个对象就行了
class FundProvider:
    def __init__(self, providerID, cashAmount, availableStartDate, availableEndDate, positionInfoList = []):
        self.providerID = providerID
        self.cashAmount = cashAmount
        self.availableStartDate = availableStartDate
        self.availableEndDate = availableEndDate
        self.positionInfoList = positionInfoList
    def updatePositionInfoList(self, newPositionInfoList):
        self.positionInfoList = newPositionInfoList

#每个PositionInfo的对象，记录持仓的每一只股票的相关信息与状态，此对象的集合会以list的方式存在ManagementAccount对象里
class PositionInfo:
    def __init__(self, positionID, tickerSymbol, marketPrice, totalValue, amount, status, dailyReturn):
        self.positionID = positionID
        self.tickerSymbol = tickerSymbol
        self.marketPrice = marketPrice
        self.totalValue = totalValue
        self.amount = amount
        self.status = status
        self.dailyReturn = dailyReturn
        
    def updateMarketPrice(self, newMarketPrice):
        self.marketPrice = newMarketPrice       
    def updatetotalValue(self, newTotalValue):
        self.totalValue = newTotalValue
    def updateAmount(self, updateAmount):
        self.amount = self.amount + updateAmount
    def updateStatus(self, newStatus):
        self.status = newStatus 
    def updateDailyReturn(self, newDailyReturn):
        self.dailyReturn = newDailyReturn

#此类模拟交易的账户，回测过程中，每个交易日生成一个账户的copy，来记录账户在每个交易日的时点信息
class ManagementAccount:
    def __init__(self, tradeDate, dailyReturn, netValue, totalAsset, positionInfoList = []):
        self.tradeDate = tradeDate
        self.dailyReturn = dailyReturn
        self.netValue = netValue 
        self.totalAsset = totalAsset
        self.positionInfoList = positionInfoList        
    def updateTradeDate(self, newTradeDate):
        self.tradeDate = newTradeDate
    def updateDailyReturn(self, newDailyReturn):
        self.dailyReturn = newDailyReturn
    def updateNetValue(self, newNetValue):
        self.netValue = newNetValue
    def updateTotalAsset(self, newTotalAsset):
        self.totalAsset = newTotalAsset
      
#此对象描述交易这个动作                       
class Trade:
    def __init__(self, tradeID, tickerSymbol, tradeDate, tradeType, tradeAmount):
        self.tradeID = tradeID
        self.tickerSymbol = tickerSymbol
        self.tradeDate = tradeDate
        self.tradeType = tradeType
        self.tradeAmount = tradeAmount        
    def setTradeAmount(self, tradeAmount):
        self.tradeAmount = tradeAmount
 

#回测过程
def backtestProcess(backtestInputDf, backtestSymbolDailyReturnList): 
    #backtestInputDf来自在线学习的输出，backtestSymbolDailyReturnList是每日每个股票的收益率的dataframe

伪代码：
    for tradeDate in tradeDateList:(循环遍历每一天)
        接收在线学习模型算出来的当日关于每个股票的推荐持仓权重（backtestInputDf）
        根据managementAccount里面的现有持仓信息挨个判断：
            如果该券可以买入（不停牌，不跌停（一字跌停？）），则做卖出操作：trade(sell), 调仓到backtestInputDf推荐的权重；
            如果该券可以买入（不停牌，不涨停（一字涨停？）），则做买入操作：trade(buy), 调仓到backtestInputDf推荐权重；
            否则，暂不对该券做买卖
        在该日内，遍历完所有券后，更新该日的managementAccount的信息，然后copy该对象，放入managementAccountList里，来记录该日的时点信息
    
    return managementAccountList, buyTradeList, sellTradeList

     
backtestResult = backtestProcess(backtestInputDf,backtestSymbolDailyReturnList)  





#以下是以前使用过的回测函数的实例，仅供参考
"""
def backtestProcess(fundProviderList, backtestInputDf, backtestSymbolDailyReturnList):
    #backtestStartDate = fundProviderList[0].availableStartDate
    #backtestEndDate = dt.date.today()
    accountPositionInfoList = []
    buyTradeList = []
    sellTradeList = []
    managementAccount = ManagementAccount(cal_start_date,0,1.0,0,accountPositionInfoList)
    managementAccountList = []
    for tradeDate in calendar['calendar_date'].values:
        for provider in fundProviderList:
            if tradeDate - provider.availableStartDate >= dt.timedelta(0) and tradeDate - provider.availableEndDate < dt.timedelta(0):
                if provider.positionInfoList == []:
                    newPositionSymbolList = list(backtestInputDf[backtestInputDf['trade_date'] == tradeDate]['ticker_symbol'].unique())
                    providerPositionInfoList = []
                    for symbolItem in newPositionSymbolList:
                        initMarketPrice = backtestSymbolDailyReturnList[symbolItem][backtestSymbolDailyReturnList[symbolItem]['end_date'] == tradeDate]['adj_nav'].values
                        if len(initMarketPrice) > 0:
                            initMarketPrice = initMarketPrice[0]
                            positionID = str(provider.providerID) + str(symbolItem)
                            initValue = provider.cashAmount/len(newPositionSymbolList)
                            positionInfo = PositionInfo(positionID, symbolItem, initMarketPrice, initValue, initValue/initMarketPrice, 1,0.0)
                            providerPositionInfoList.append(positionInfo)
                            trade = Trade(positionID, symbolItem, tradeDate, 1, initValue/initMarketPrice)
                            buyTradeList.append(trade)
                    provider.updatePositionInfoList(providerPositionInfoList)
                    
                else:
                    for positionItem in provider.positionInfoList:
                        if positionItem.status == 0:
                            continue
                        symbolItem = positionItem.tickerSymbol
                        marketPrice = backtestSymbolDailyReturnList[symbolItem][backtestSymbolDailyReturnList[symbolItem]['end_date'] == tradeDate]['adj_nav'].values 
                        if len(marketPrice) > 0:
                            marketPrice = marketPrice[0]
                            amount = positionItem.amount
                            positionItem.updateDailyReturn(marketPrice/positionItem.marketPrice - 1)
                            positionItem.updateMarketPrice(marketPrice)
                            positionItem.updatetotalValue(marketPrice*amount)  
                        else:
                            positionItem.updateDailyReturn(0)
                            positionItem.updateStatus(0)
                            trade = Trade(positionItem.positionID, positionItem.tickerSymbol, tradeDate, -1, positionItem.amount)
                            sellTradeList.append(trade)
            
            elif tradeDate - provider.availableEndDate >= dt.timedelta(0):
                if provider.positionInfoList != []:
                    for positionItem in provider.positionInfoList:
                        if positionItem.status == 1:
                            symbolItem = positionItem.tickerSymbol
                            marketPrice = backtestSymbolDailyReturnList[symbolItem][backtestSymbolDailyReturnList[symbolItem]['end_date'] == tradeDate]['adj_nav'].values
                            if len(marketPrice) > 0:
                                marketPrice = marketPrice[0]
                                amount = positionItem.amount
                                positionItem.updateDailyReturn(marketPrice/positionItem.marketPrice - 1)
                                positionItem.updateMarketPrice(marketPrice)
                                positionItem.updatetotalValue(marketPrice*amount)
                            else:
                                positionItem.updateDailyReturn(0)
                            positionItem.updateStatus(0)
                            trade = Trade(positionItem.positionID, positionItem.tickerSymbol, tradeDate, -1, positionItem.amount)
                            sellTradeList.append(trade)
                  
        managementAccount.updateTradeDate(tradeDate)
        dailyReturn = 0
        totalAsset = 0
        for provider in fundProviderList:
            for positionItem in provider.positionInfoList:
                dailyReturn += positionItem.dailyReturn*positionItem.totalValue
                totalAsset += positionItem.totalValue
        if totalAsset > 1e-10:
            managementAccount.updateDailyReturn(dailyReturn/totalAsset)
        else:
            managementAccount.updateDailyReturn(0.0)
        managementAccount.updateNetValue((managementAccount.dailyReturn + 1)*managementAccount.netValue)
        managementAccount.updateTotalAsset(totalAsset)
         
        managementAccountCopy = copy.deepcopy(managementAccount)
        #将当天的account对象复制，并且加入list：
        managementAccountList.append(managementAccountCopy)
        print(tradeDate)
    return managementAccountList,buyTradeList,sellTradeList
"""